In [1]:
import pandas as pd
import numpy as np

In [2]:
client = "stadium"
df = pd.read_excel(f"./{client}_results.xlsx")
df

,hostname,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration,Client Total Fixation Duration,Sample Size
0,aftonbladet.se,1369241,995175,349022,35.07,2367.65,123118,8.263619e+08,high
1,malservice.aftonbladet.se,5,2,0,0.00,NaN,0,NaN,low
2,svd.se,7807,5068,2787,54.99,3301.47,198088,9.201197e+06,medium
3,tv.nu,9800,6031,1782,29.55,985.25,3941,1.755716e+06,medium


In [3]:
df = df[df["Sample Size"].isin(["high", "medium"])]
df["group"] = 1

/tmp/ipykernel_22833/497368128.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["group"] = 1


In [4]:
final_df = df.groupby(["group"]).apply(
    lambda x: pd.Series(
        {
            "Impressions": np.floor(np.average(x["Impressions"], weights=x["Impressions"])),
            "Inviews": np.floor(np.average(x["Inviews"], weights=x["Inviews"])),
            "Fixations": np.floor(np.average(x["Fixations"], weights=x["Fixations"])),
            "Fixation/Inview Ratio": round(np.average(x["Fixation/Inview Ratio"], weights=x["Impressions"]), 2),
            "Average Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"])),
            "Total Fixation Duration": np.floor(np.average(x["Average Fixation Duration"], weights=x["Fixations"]) *  np.floor(np.average(x["Fixations"], weights=x["Fixations"])))
        }
    )
)
final_df = final_df.reset_index()
final_df

,group,Impressions,Inviews,Fixations,Fixation/Inview Ratio,Average Fixation Duration,Total Fixation Duration
0,1,1351970.0,984260.0,344542.0,35.14,2368.0,815890425.0


In [5]:
final_df = final_df[["Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration"]]

final_df.to_excel(f"./{client}_results_consolidated.xlsx", index=False)